In [1]:
import numpy as np
import pandas as pd
import math

Determine CDS Spread

In [2]:
'''
R = Risk Free Rate
T = Time to maturity in years
dt = Payment Frequency in fraction of a year  .25 = quarterly, .5 = semiannually
N = Notional Amount
Lam = Hazard Rate
RR = Recovery Rate
'''
R = .05
T = 5
dt = .25
N = 1000000
Lam = .03
RR = .40

In [4]:
#Create empty data frame and create index by period
df = pd.DataFrame()
index = np.arange(0,T+dt,dt)
df['period'] = index
df = df.set_index(index)

#Set initial variables in the dataframe
df['lambda'] = Lam
df['Notional'] = N

#Calculate probability of survival and probability of default for given hazard rate
df['P_Survival'] = np.exp(df['period']*-df['lambda'])
df['P_Default'] = df['P_Survival'].shift(1) - df['P_Survival']
df.loc[0,'P_Default'] = 0

#Calculate discount factors for cash flows
df['disc_factor'] = np.exp(-R * df['period'])

#Calculate the cash flows related to the premium and default legs of the CDS
df['premium_leg'] = df['Notional'] * df['disc_factor'] * df['P_Survival'] * dt
df.loc[0,'premium_leg'] = 0
df['default_leg'] = df['Notional'] * (1-RR) * df['P_Default'] * df['disc_factor']

pv_premium_leg = df['premium_leg'].sum()
print("PV of Premium Leg = {:,.2f}".format(pv_premium_leg))
pv_default_leg = df['default_leg'].sum()
print("PV of Default Leg = {:,.2f}".format(pv_default_leg))

premium = pv_default_leg / pv_premium_leg
print("Premium - CDS Spread = %.4f" %premium)

PV of Premium Leg = 4,079,926.80
PV of Default Leg = 73,714.77
Premium - CDS Spread = 0.0181
